# 3.Data Visualization and Insights
1. Firms Part
2. Population Part
3. Salary Part 
4. Geography Part
5. Heatmap Analysis

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
# Press "shift + Enter" to run the cell in .ipynb file

import matplotlib.pyplot as plt # 2D plotting library
import seaborn as sns # a high-level plotting library
import warnings
warnings.simplefilter(action='ignore') # stop warnings 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [1]:
data2 = pd.read_csv("../input/cleaned-data/Final_Data.csv")

1. Firms Part

In [1]:
# The distribution of total number of firms in town
sns.distplot(data2.total_firms)

- From the distribution of total number of firms in town, we can find that most of firms are not in a very big scale, so we need to foucus on the firms with relative low number. 

In [1]:
data2.total_firms.quantile(0.9)

- Since the number above 1000 is belong to Large firms we have defined, then we choose 90% of them as our target.

###  In conclusion, most of firms' scale are 'Small' and 'Medium', with number around 200.

In [1]:
target = data2[data2.total_firms < data2.total_firms.quantile(0.9)]
sns.distplot(target.total_firms,axlabel='Total Number of Firms')

- For the rest of firms with large scale, we anlyze top 5 of them.`

In [1]:
top10_firms = data2.nlargest(5,'total_firms')
top10_firms = top10_firms.reset_index()
top10_firms[['Town','total_firms']]

In [1]:
top10_firms_type = top10_firms.iloc[:,-4:]
top10_firms_type.columns = columns=['Micro','Small','Medium','Large']
top10_firms_type.index = index=top10_firms.Town
top10_firms_type.plot.bar(rot=0)

### Paris is the central of France with 427K firms, almost 6 times than second city Marseille(68K firms). Micro-scale company is the main type of company in all top cities.

- So, we want to check whether micro scale company is the main type of city in all regions. We use Micro_firms column to compare with other types of firms

In [1]:
a = 0
for i in range(len(data2)):
    if data2.iloc[i,-4]<= (max(data2.iloc[i,-3],data2.iloc[i,-2],data2.iloc[i,-1])):
        a += 1
        print('Other scale company is equal or larger than Microcompany in',data2['Town'][i])
print('The number of cities that micro scale firms is not main kind of company: ',a)

In [1]:
data2.iloc[:,[2,-4,-3,-2,-1]][data2.Town.isin(
    ['Brenouille','Bihorel','Le Plessis-Pâté'])]

- Bihorel do not have any kinds of comapany at all, so we exclude it.

### Except Brenouille and Le Plessis-Pâté, Micro_firms are the main type of company in all cities.

2. Population Part

In [1]:
sns.distplot(data2.total_population)

- In order to avoid the impact of the maximum value, we also extract 90% quantile of data to explore the population distribution of all regions.

In [1]:
target2 = data2[data2.total_population < data2.total_population.quantile(0.9)]
sns.distplot(target2.total_population,axlabel='Total Population')

- Next, we want to find out whether the regional population will affect the concentration of the company. We also choose top 5 high population city

In [1]:
top10_population = data2.nlargest(5,'total_population')
top10_population = top10_population.reset_index()
top10_population[['Town','total_population']]

- Higher population indeed contribute to higher numbers of firms in those top cities.So, we want to explore more on the correlation between population and number of firms.

In [1]:
data2['total_population'].corr(data2['total_firms'])

### Population has high correlation to number of firms with 0.94 correlation coefficient. Large population make bigger market, more options, more opportunities for more business development.

3. Salary Part

- To begin with, we observe the overrall distribution information of mean_salary.

In [1]:
data2.mean_salary.describe()

### Inequality salary phenomenon is obvious in France, the maximum salary is almost 4 times than minimum salary and 3 times than mean salary.

- From sex aspect, we found men tend to have higher salary than women. Then we try to prove it works in other aspects.

In [1]:
sns.barplot(x=['mean_female_salary','mean_male_salary'],y=[data2.mean_female_salary.mean(),data2.mean_male_salary.mean()])

- We can see in all age groups man receive higher salary than women, with a larger difference for older age groups.

In [1]:
age_groups = pd.DataFrame([[data2.mean_young_female_salary.mean(),data2.mean_young_male_salary.mean()],
                   [data2.mean_medium_female_salary.mean(),data2.mean_medium_male_salary.mean()],
                   [data2.mean_old_female_salary.mean(),data2.mean_old_male_salary.mean()]],
                  columns=['female','male'],index=['young','medium','old'])
age_groups.plot.bar(rot=0)

- We can also observe that in all job categories, man gain more salary than women, with a higher difference for higher positions.

In [1]:
job_category = pd.DataFrame([[data2.mean_female_employee_salary.mean(),data2.mean_male_employee_salary.mean()],
                             [data2.mean_female_worker_salary.mean(),data2.mean_male_worker_salary.mean()],
                            [data2.mean_female_middle_manager_salary.mean(),data2.mean_male_middle_manager_salary.mean()],
                            [data2.mean_female_executive_salary.mean(),data2.mean_male_executive_salary.mean()]],
                            columns=['female','male'],index=['employee','worker','middle_manager','executive'])
job_category.plot.bar(rot=0)

### In conclusion, gender affects salary acquisition, men always recieve more in all age groups and all job categories.

4. Geography Part

- To get a more clear and interactive version of map analysis, we use Tableau to visualize our geography data. Here is my map analysis Link: https://public.tableau.com/profile/zhimao.xie#!/vizhome/FranceSalaryDistribution/FranceSalaryDistribution 
- Users can choose the salary and population combination by using the slide bars above.

In [1]:
from IPython.display import Image
Image("../input/map-image/Map.PNG")

5. Heatmap Analysis

- To prove conclusions we made, we adopt heatmap to show the correlation between our Target_features.

In [1]:
Target_fratures = data2[['mean_salary','total_firms','total_population','mean_executive_salary', 'mean_middle_manager_salary',
       'mean_employee_salary', 'mean_worker_salary','mean_male_salary','mean_female_salary',
                 'mean_young_age_salary','mean_medium_age_salary', 'mean_old_age_salary']]
corr = Target_fratures.corr()
plt.figure(figsize=(14,14))
sns.heatmap(corr, annot=True)
plt.show()

### Apart from conclusions we made, we also can see an interesting phenomenon that young_age_salary has relative lower correlation with other features than other groups of salary. Because young people are full of more uncertainty, their average wages are more unstable.

# 4.Conclusion

1. From Firms Dimension:
   1. Most of firms' scale are 'Small' and 'Medium', with number around 200.
   2. Paris is the central of France with 427K firms, almost 6 times than second city Marseille(68K firms). 
   3. Except Brenouille and Le Plessis-Pâté, Micro_firms are the main type of company in all cities.
2. From Population Dimension:
   1. Population has high correlation to number of firms with 0.94 correlation coefficient. Large population make bigger market, more options, more opportunities for more business development.
3. From Salary Dimension:
   1. Inequality salary phenomenon is obvious in France, the maximum salary is almost 4 times than minimum salary and 3 times than mean salary.
   2. Gender affects salary acquisition, men always recieve more in all age groups and all job categories.
   3. Young_age_salary has relative lower correlation with other groups of salary. Because young people are full of more uncertainty, their average wages are more unstable.
4. From Geography Dimension:
   1. There is no obivous correlation between Population Distribution and Salary Distributinon in France. Some relative small population Towns can still have higher mean salary than Paris. 
   2. Paris did play a key role as Economic center, most of towns with higher salary concentrate nearby Paris.

# Thank You So Much~